### Extract Sample Images for Point Cloud Algorithm
Get images from the "imgs" folder for 3D reconstruction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import shutil
import glob
import cv2

# Configuration
imgs_dir = Path('./imgs')
output_dir = Path('./sample_images')
output_dir.mkdir(exist_ok=True)

# Get all images from imgs folder
image_files = sorted(glob.glob(str(imgs_dir / '*.png')))
num_samples = min(5, len(image_files))  # Use up to 5 images

print(f"=== Loading {num_samples} Images from imgs/ folder ===\n")
print(f"Found {len(image_files)} total images\n")

# Store image data (no camera poses available for these images)
sample_data = []

for i, img_path in enumerate(image_files[:num_samples]):
    img_path = Path(img_path)
    
    # Copy to output directory with standardized name
    output_path = output_dir / f"image_{i}.png"
    shutil.copy(img_path, output_path)
    
    # Store info
    sample_data.append({
        'index': i,
        'original_name': img_path.name,
        'color_path': output_path,
        'pose_matrix': None  # No poses available for custom images
    })
    
    print(f"✓ Image {i}: {img_path.name}")
    print(f"  Saved to: {output_path}")

print(f"\n✓ All images copied to: {output_dir}/")
print(f"\nYou now have:")
print(f"  - {num_samples} RGB images (image_0.png to image_{num_samples-1}.png)")
print(f"\n⚠ Note: No camera poses available - will estimate from images using SfM")

### Next Steps: Point Cloud Reconstruction

Images loaded from custom folder - camera poses will be estimated!

**Reconstruction Pipeline:**
1. Extract SIFT features from each image
2. Match features between consecutive image pairs
3. Estimate camera poses using Essential Matrix
4. Triangulate 3D points from correspondences
5. Build colored point cloud

**Camera Intrinsics:**
- Estimated from image dimensions
- Assumes typical consumer camera
- Focal length ≈ 1.2 × max(width, height)

For better results, consider camera calibration first!

In [ ]:
# Check if we have camera poses
has_poses = sample_data[0]['pose_matrix'] is not None

if has_poses:
    # Display camera pose information
    print("=== Camera Pose Matrices ===\n")
    print("Each 4x4 matrix represents camera position and orientation in 3D space")
    print("Format: [R | t] where R is 3x3 rotation, t is 3x1 translation\n")

    for i, data in enumerate(sample_data):
        print(f"--- Image {i} (frame {data['frame']}) ---")
        print(data['pose_matrix'])
        print(f"Camera position (x, y, z): [{data['pose_matrix'][0,3]:.3f}, {data['pose_matrix'][1,3]:.3f}, {data['pose_matrix'][2,3]:.3f}]")
        print()
else:
    print("=== No Camera Poses Available ===\n")
    print("Using custom images from imgs/ folder.")
    print("Camera poses will need to be estimated using Structure from Motion (SfM).")
    print("\nFor point cloud reconstruction, we'll use:")
    print("  1. Feature matching between images")
    print("  2. Essential matrix estimation")
    print("  3. Camera pose recovery")
    print("  4. Triangulation of 3D points")

In [ ]:
# Visualize the selected images
fig, axes = plt.subplots(1, len(sample_data), figsize=(4*len(sample_data), 4))

# Handle case where there's only 1 image
if len(sample_data) == 1:
    axes = [axes]

for i, data in enumerate(sample_data):
    img = Image.open(data['color_path'])
    axes[i].imshow(img)
    title = f"Image {i}\n{data['original_name']}"
    axes[i].set_title(title)
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(output_dir / 'sample_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved visualization to sample_images/sample_overview.png")

In [ ]:
# Save point cloud to file
output_ply = output_dir / 'point_cloud.ply'

# Write PLY file
with open(output_ply, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write(f"element vertex {len(point_cloud)}\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("property uchar red\n")
    f.write("property uchar green\n")
    f.write("property uchar blue\n")
    f.write("end_header\n")
    
    for i in range(len(point_cloud)):
        x, y, z = point_cloud[i]
        r, g, b = (point_colors[i] * 255).astype(int)
        f.write(f"{x} {y} {z} {r} {g} {b}\n")

print(f"✓ Point cloud saved to: {output_ply}")
print(f"\nPoint cloud statistics:")
print(f"  Total points: {len(point_cloud)}")
print(f"  X range: [{point_cloud[:, 0].min():.3f}, {point_cloud[:, 0].max():.3f}]")
print(f"  Y range: [{point_cloud[:, 1].min():.3f}, {point_cloud[:, 1].max():.3f}]")
print(f"  Z range: [{point_cloud[:, 2].min():.3f}, {point_cloud[:, 2].max():.3f}]")
print(f"\nYou can open the .ply file in MeshLab, CloudCompare, or Blender!")

In [ ]:
# Visualize the point cloud
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Downsample for visualization
max_points = 10000
if len(point_cloud) > max_points:
    indices = np.random.choice(len(point_cloud), max_points, replace=False)
    plot_points = point_cloud[indices]
    plot_colors = point_colors[indices]
else:
    plot_points = point_cloud
    plot_colors = point_colors

# Plot
ax.scatter(plot_points[:, 0], plot_points[:, 1], plot_points[:, 2],
           c=plot_colors, marker='.', s=1, alpha=0.6)

ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
ax.set_zlabel('Z (meters)')
ax.set_title(f'3D Point Cloud from {len(sample_data)} Images\n({len(point_cloud)} points)')

# Set equal aspect ratio
max_range = np.array([plot_points[:, 0].max() - plot_points[:, 0].min(),
                      plot_points[:, 1].max() - plot_points[:, 1].min(),
                      plot_points[:, 2].max() - plot_points[:, 2].min()]).max() / 2.0

mid_x = (plot_points[:, 0].max() + plot_points[:, 0].min()) * 0.5
mid_y = (plot_points[:, 1].max() + plot_points[:, 1].min()) * 0.5
mid_z = (plot_points[:, 2].max() + plot_points[:, 2].min()) * 0.5

ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

plt.savefig(output_dir / 'point_cloud.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Point cloud visualization saved to {output_dir / 'point_cloud.png'}")

In [ ]:
# Build point cloud from all image pairs
print("=== Building Point Cloud ===\n")

all_points_3d = []
all_colors = []

has_poses = sample_data[0]['pose_matrix'] is not None

# First camera is at origin
if not has_poses:
    # Initialize first pose as identity (at origin)
    sample_data[0]['pose_matrix'] = np.eye(4)

# Process consecutive image pairs
for i in range(len(sample_data) - 1):
    img1_path = sample_data[i]['color_path']
    img2_path = sample_data[i + 1]['color_path']
    
    # Load images
    img1 = np.array(Image.open(img1_path))
    img2 = np.array(Image.open(img2_path))
    
    print(f"Processing pair: Image {i} <-> Image {i+1}")
    
    # Extract and match features
    pts1, pts2, matches = extract_and_match_features(img1, img2)
    
    if len(pts1) < 8:
        print(f"  ⚠ Not enough matches ({len(pts1)}), skipping")
        continue
    
    print(f"  Found {len(matches)} good matches")
    
    if has_poses:
        # Use ground truth poses
        pose1 = sample_data[i]['pose_matrix']
        pose2 = sample_data[i + 1]['pose_matrix']
    else:
        # Estimate relative pose using Essential matrix
        E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, R, t, mask = cv2.recoverPose(E, pts1, pts2, K)
        
        # Build pose matrix for second camera
        pose1 = sample_data[i]['pose_matrix']
        pose2 = np.eye(4)
        pose2[:3, :3] = R
        pose2[:3, 3] = t.flatten()
        
        # Compose with previous pose (chain transformations)
        sample_data[i + 1]['pose_matrix'] = pose2 @ pose1
        
        print(f"  Estimated relative pose:")
        print(f"    Rotation magnitude: {np.linalg.norm(R - np.eye(3)):.3f}")
        print(f"    Translation: [{t[0,0]:.3f}, {t[1,0]:.3f}, {t[2,0]:.3f}]")
    
    # Get final poses
    pose1 = sample_data[i]['pose_matrix']
    pose2 = sample_data[i + 1]['pose_matrix']
    
    # Create projection matrices: P = K * [R | t]
    P1 = K @ pose1[:3, :]  # 3x4 projection matrix
    P2 = K @ pose2[:3, :]
    
    # Triangulate points
    points_3d = triangulate_points(pts1, pts2, P1, P2)
    
    # Get colors from first image
    colors = []
    for pt in pts1:
        x, y = int(pt[0]), int(pt[1])
        if 0 <= x < img1.shape[1] and 0 <= y < img1.shape[0]:
            colors.append(img1[y, x] / 255.0)
        else:
            colors.append([0.5, 0.5, 0.5])
    
    colors = np.array(colors)
    
    # Filter outliers (points too far from camera)
    valid_mask = np.abs(points_3d).max(axis=1) < 10.0  # within 10 meters
    points_3d_filtered = points_3d[valid_mask]
    colors_filtered = colors[valid_mask]
    
    print(f"  Triangulated {len(points_3d_filtered)} valid 3D points")
    
    all_points_3d.append(points_3d_filtered)
    all_colors.append(colors_filtered)

# Combine all points
if len(all_points_3d) > 0:
    point_cloud = np.vstack(all_points_3d)
    point_colors = np.vstack(all_colors)
    print(f"\n✓ Total point cloud size: {len(point_cloud)} points")
else:
    print("\n✗ No points generated. Check image matching quality.")
    point_cloud = np.zeros((0, 3))
    point_colors = np.zeros((0, 3))

In [ ]:
def triangulate_points(pts1, pts2, P1, P2):
    """Triangulate 3D points from 2D correspondences and projection matrices"""
    # Convert to homogeneous coordinates
    pts1_norm = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K, None)
    pts2_norm = cv2.undistortPoints(pts2.reshape(-1, 1, 2), K, None)
    
    # Triangulate
    points_4d = cv2.triangulatePoints(P1, P2, pts1_norm, pts2_norm)
    
    # Convert from homogeneous to 3D
    points_3d = points_4d[:3] / points_4d[3]
    
    return points_3d.T

print("Triangulation function defined")

In [ ]:
def extract_and_match_features(img1, img2):
    """Extract SIFT features and match between two images"""
    # Convert to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    # Create SIFT detector
    sift = cv2.SIFT_create(nfeatures=2000)
    
    # Detect and compute
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)
    
    # Match features using FLANN
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    matches = flann.knnMatch(des1, des2, k=2)
    
    # Apply Lowe's ratio test
    good_matches = []
    pts1 = []
    pts2 = []
    
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
            pts1.append(kp1[m.queryIdx].pt)
            pts2.append(kp2[m.trainIdx].pt)
    
    return np.float32(pts1), np.float32(pts2), good_matches

print("Feature extraction and matching function defined")

In [ ]:
import cv2
import numpy as np

# Load first image to get dimensions
first_img = Image.open(sample_data[0]['color_path'])
img_width, img_height = first_img.size

# Estimate camera intrinsics (typical for consumer cameras)
# Focal length estimated as 1.2 * max(width, height)
focal_length = 1.2 * max(img_width, img_height)
cx, cy = img_width / 2.0, img_height / 2.0  # principal point at center

K = np.array([
    [focal_length, 0, cx],
    [0, focal_length, cy],
    [0, 0, 1]
])

print("=== Camera Intrinsic Matrix (Estimated) ===")
print(K)
print(f"\nImage resolution: {img_width}x{img_height}")
print(f"Estimated focal length: {focal_length:.1f}px")
print(f"Principal point: ({cx:.1f}, {cy:.1f})")
print("\nNote: These are typical estimates. Actual calibration would improve results.")

### Create Point Cloud from Images
Using feature matching and triangulation with known camera poses